# Simulated X-ray Spectrum of Gas in CIE

Simulated X-ray spectrum of solar abundance low-density ($n_e$=0.004), hot (T=10$^6$K) gas in collisional
ionization equilibrium (CIE). Ionic species responsible for various emission lines are labeled. Wavelengths
range from 150 to 250 Angstroms in steps of 0.5 Angstroms.

This notebook uses ChiantiPy to compute the spectrum data plotted by Fig5_8_XRaySpectrum.ipynb.  This notebook
may be used, for example, to explore other values of temperature, density, and emission measure
than used for Figure 5.8.

To keep you from accidentally ovewriting the data file used by the notebook for Figure 5.8, we've named the
output spectrum file "mySpectrum.txt".

You will need to install the latest versions of ChiantiPy and the CHIANTI atomic database - available separately -
before you can use this notebook, and the versions of each must be compatible. At the time of this notebook's
release, we tested it with ChiantiPy version 0.9.5 and CHIANTI version 10.0. See the caveat below.

**This notebook only computes the spectrum, it does not plot it.**  The calculation takes many minutes to 
complete.


## Caveat

ChiantiPy and CHIANTI must both be installed on your system and uptodate to run this notebook. Exceptions
raised because of missing files are often due to having updated ChiantiPy but not the full CHIANTI database.
Being diligent about keeping the two in-sync is part of using Chianti.

### CHIANTI citation

CHIANTI is a collaborative project involving George Mason University, the University of Michigan (USA),
University of Cambridge (UK) and NASA Goddard Space Flight Center (USA). 

In [3]:
import math
import numpy as np

# Throttle nuisance warnings

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)

# Chianti CIE code 

import ChiantiPy.core as ch
import ChiantiPy.tools.filters as cf

## Calculate a simulated spectrum

Compute the X-ray spectrum of a CIE plasma with these parameters:
 * Temperature: T=10<sup>6</sup>
 * Electron density: n$_e$=0.004 cm$^{-3}$
 * Emission Measure: EM=10<sup>25</sup>.

The spectrum is calculated for wavelengths from 150 to 270 Angstroms in 0.5 Angstrom steps.

The minimum abundance species to use for the calculation is set to **minAbund=2e-5**, which adds N, Mg, Si,
S, and Fe to the calculation (set this to 1e-4 to include just H, He, C, O, and Ne).  Setting minAbund
to 1e-6 will add Na, Al, Ar, Ca, and Ni, but the computation time required can be very long.

Elapsed computation time with minAbund=2e-5 was ~200 seconds for us, your performance will likely differ.

In [4]:
# plasma parameters

T = 1.0e+6   # plasma temperature in Kelvin
ne = 0.004   # electron density in cm^-3, typical of the local HIM bubble
em = 1.0e+25 # emission measure 

# Spectrum parameters

minLam = 150.0 # starting wavelength in Angstroms
maxLam = 270.0 # ending wavelength in Angstroms
lamStep = 0.5  # wavelength step in Angstroms
width = lamStep # Gaussian FWHM in Angstroms

# Set minAbund low enough to include every species in the database

minAbund = 2.e-5

# Do the computation...

numLam = 1+int((maxLam-minLam)/lamStep)
lam = np.linspace(minLam,maxLam,numLam)

spec = ch.spectrum(T,ne,lam,filter=(cf.gaussian,width),em=em,minAbund=minAbund)

# some individual ions

fe9 = ch.ion('fe_9',temperature=T,eDensity=ne)
fe9.spectrum(lam,filter=(cf.gaussian,width))

fe10 = ch.ion('fe_10',temperature=T,eDensity=ne)
fe10.spectrum(lam,filter=(cf.gaussian,width))

fe11 = ch.ion('fe_11',temperature=T,eDensity=ne)
fe11.spectrum(lam,filter=(cf.gaussian,width))

fe12 = ch.ion('fe_12',temperature=T,eDensity=ne)
fe12.spectrum(lam,filter=(cf.gaussian,width))

s9 = ch.ion('s_9',temperature=T,eDensity=ne)
s9.spectrum(lam,filter=(cf.gaussian,width))

si9 = ch.ion('si_9',temperature=T,eDensity=ne)
si9.spectrum(lam,filter=(cf.gaussian,width))

he2 = ch.ion('he_2',temperature=T,eDensity=ne)
he2.spectrum(lam,filter=(cf.gaussian,width))


 elapsed seconds =      212.000


## Create an ASCII file with the spectrum

Multicolumn ASCII-text file with the calculated X-ray spectrum.

In [9]:
specFile = 'mySpectrum.txt'

out = open(specFile,'w')
out.write(f'#\n')
out.write(f'# CIE spectrum logT={math.log10(T):.1f}K ne={ne:.3f}/cc EM={em:.1e}\n')
out.write(f'# Calculated using CHIANTI and ChiantiPy\n')
out.write(f'#\n')

out.write('lam     total  fe9    fe10   fe11   fe12   s9     si9    he2\n')

for i, l in enumerate(lam):
    outStr = '%6.2f %6.3f %6.3f %6.3f %6.3f %6.3f %6.3f %6.3f %6.3f\n' % (l,
                                                                 spec.Spectrum['integrated'][i],
                                                                 fe9.Spectrum['intensity'][i]*em,
                                                                 fe10.Spectrum['intensity'][i]*em,
                                                                 fe11.Spectrum['intensity'][i]*em,
                                                                 fe12.Spectrum['intensity'][i]*em,
                                                                 s9.Spectrum['intensity'][i]*em,
                                                                 si9.Spectrum['intensity'][i]*em,
                                                                 he2.Spectrum['intensity'][i]*em)

    out.write(outStr)

out.close()